In [77]:
using Oscar

In [78]:
# Defined the denominator
sl2size(p, k) = p^(2*k - 2) * (p^2 - 1)

sl2size (generic function with 1 method)

In [79]:
# this function generate all the matrices satisfying the condition of ratio 1
# uses dictionary to store the result so that it is easier to produce the reduced form
function build_matrices(p, n, trace, det)
    Rn, = residue_ring(ZZ, p^n)
    result = Dict{NTuple{3, zzModRingElem}, Int}()

    for a in Rn
        rhs = Rn(a*(trace - a) - det)
        for b in Rn
            if lift(b) % p != 0
                result[(a, b, rhs/b)] = 1
            end
        end

        if rhs != 0
            rhs_lift = lift(rhs)
            pval = valuation(rhs_lift, p)
            for exp in 1:pval
                rhs_new = Rn(rhs_lift/p^exp)
                for j in 1:(p^(n-exp))-1
                    if j % p != 0
                        b = Rn(p^exp * j)
                        c_start = lift(rhs_new/j) % p^(n-exp)
                        for c in c_start: p^(n-exp): p^n-1
                            result[(a, b, Rn(c))] = 1
                        end
                    end
                end
            end
        else
            for c in Rn
                result[(a, Rn(0), c)] = 1
            end

            for exp in 1:n-1
                for j in 1:p^(n-exp)-1
                    if j % p != 0
                        b = Rn(p^exp * j)
                        for c in 0:p^(n-exp):p^n-1
                            result[(a, b, Rn(c))] = 1
                        end
                    end
                end
            end
        end
    end

    return result
end

build_matrices (generic function with 1 method)

In [80]:
# given a set of vectors, try to find a matrix A with minimum vp(det(A)) by trying the linear combination of the vectors
function det_check(p, k, ker, Rn)
    minVal = 20
    l = size(ker, 2)
    ker_cols = [(Rn(ker[1, j]), Rn(ker[2, j]), Rn(ker[3, j]), Rn(ker[4, j])) for j in 1:l]
    for i in 1:l
        (a1, a2, a3, a4) = ker_cols[i]
        for i1 in 0:1, i2 in 0:1, i3 in 0:1, i4 in 0:1
            det = (a1+i1*p^k) * (a4+i4*p^k) - (a2+i2*p^k) * (a3+i3*p^k)
            if det != 0 && valuation(lift(det), p) < minVal
                minVal = valuation(lift(det), p)
            end
        end
        for j in (i+1):l
            b1, b2, b3, b4 = ker_cols[j]
            det = (a1+b1)*(a4+b4) - (a2+b2)*(a3+b3)
            if det != 0 && valuation(lift(det), p) < minVal
                minVal = valuation(lift(det), p)
            end
        end
    end
    return minVal
end

det_check (generic function with 1 method)

In [81]:
function ratio3(p, n, k, equations_template, matrices)
    Rk, = residue_ring(ZZ, p^k)
    Rn, = residue_ring(ZZ, p^n)
    ker_hist = Dict{zzModMatrix, Int64}()
    res = Dict{Tuple{zzModRingElem, zzModRingElem, zzModRingElem}, Int64}()
    
    for (M, num) in matrices
        idx = Int(lift(M[1]))+1
        equations = equations_template[idx]
        equations[1,3], equations[3,4], equations[2,1], equations[4,2] = M[2], M[2], M[3], M[3] # only changes 4 entries of the 4x4 matrix
        ker = kernel(equations, side=:right)
        if haskey(ker_hist, ker)
            res[M] = ker_hist[ker]
        else
            minVal =  det_check(p, k, map(lift, ker), Rn)
            ker_hist[ker] = minVal
            res[M] = minVal
        end
    end
    return res
end

ratio3 (generic function with 2 methods)

In [82]:
# this function computes ratio3 for k in (1, n) and e in (1, max_e)
function compute_ratio3(p, n, baseM, max_e)
    trace, det = baseM[1]+baseM[4], baseM[1]*baseM[4] - baseM[2]*baseM[3]
    equations_template = Vector{Vector{MatElem}}(undef, n)

    # generating templates for equations in form of 4x4 matrices
    for k in 1:n
        Rk, = residue_ring(ZZ, p^k)
        templates = Vector{MatElem}(undef, p^k)
        for a in 0:(p^k - 1)
            templates[a + 1] = matrix(Rk, [
                [a - baseM[1], -baseM[3], 0, 0],
                [0, 0, (trace - a) - baseM[1], -baseM[3]],
                [-baseM[2], a - baseM[4], 0, 0],
                [0, 0, -baseM[2], (trace - a) - baseM[4]]
            ])
        end
        equations_template[k] = templates
    end

    # generating all the matrices satisfying the condition of having the same trace and determinant
    matrices = [Dict{NTuple{3, zzModRingElem}, Int}() for _ in 1:n]
    matrices[n] = build_matrices(p, n, trace, det)
    for k in (n-1):-1:1
        Rk, = residue_ring(ZZ, p^k)
        for (M, num) in matrices[k+1]
            M_reduced = map(Rk, M)
            matrices[k][M_reduced] = get(matrices[k], M_reduced, 0) + num
        end
    end
    
    ratio3_val = Dict{Int, Dict{Int, Int}}()
    for k in 1:n
        ratio3_val[k] = Dict{Int, Int}()
        for e in 0:max_e
            ratio3_val[k][e] = 0
        end
    end
    
    for k in 1:n
        res = ratio3(p, n, k, equations_template[k], matrices[k])
        for (M, minVal) in res
            for e in max_e:-1:minVal
                ratio3_val[k][e] += matrices[k][M]
            end
        end
    end
    
    ratio3_res = [[rationalize(ratio3_val[k][e] / sl2size(p, n)) for k in 1:n] for e in 0:max_e]
    println("ratio 3:")
    for (e, row) in enumerate(ratio3_res)
        println("e = $(e - 1) → ", row)
    end
end

compute_ratio3 (generic function with 2 methods)

In [83]:
@time compute_ratio3(3, 6, (1, 0, 0, 10), 5)

ratio 3:
e = 0 → Rational{Int64}[1//2, 1//6, 1//6, 1//6, 1//18, 1//54]
e = 1 → Rational{Int64}[3//2, 1//2, 1//2, 1//2, 7//18, 7//54]
e = 2 → Rational{Int64}[3//2, 3//2, 3//2, 3//2, 3//2, 7//6]
e = 3 → Rational{Int64}[3//2, 3//2, 3//2, 3//2, 3//2, 25//18]
e = 4 → Rational{Int64}[3//2, 3//2, 3//2, 3//2, 3//2, 3//2]
e = 5 → Rational{Int64}[3//2, 3//2, 3//2, 3//2, 3//2, 3//2]
  8.745168 seconds (29.69 M allocations: 3.115 GiB, 45.21% gc time, 1.81% compilation time)


In [88]:
# this series of functions works more efficiently if we want to count ratio3 for specific k < n
function add_to_dict1(p, n, k, i, rhs_new, matrices_dict, a, exp, Rk)
    for j in 1:(p^exp-1)
        if j % p != 0
            b = p^i * j
            c_start = lift(Rk(rhs_new) / j) % p^(n - i)
            for c in c_start:p^exp:p^k-1
                matrices_dict[(Rk(a), Rk(b), Rk(c))] = get(matrices_dict, (Rk(a), Rk(b), Rk(c)), 0) + p^(n-k)
            end
        end
    end
end

function add_to_dict2(p, n, k, pval, matrices_dict, a, b, val1, val2, Rk)
    if n-pval >= k
        matrices_dict[(Rk(a), Rk(b), zero(Rk))] = get(matrices_dict, (Rk(a), Rk(b), zero(Rk)), 0) + val1
    else
        for c in 0:p^(n - pval):p^k - 1
            matrices_dict[(Rk(a), Rk(b), Rk(c))] = get(matrices_dict, (Rk(a), Rk(b), Rk(c)), 0) + val2
        end
    end
end

function build_matrices_ratio3(p, n, trace, det, k)
    matrices_dict = Dict{NTuple{3, Any}, Int}()
    Rk, = residue_ring(ZZ, p^k)
    Rn, = residue_ring(ZZ, p^n)

    for a in 0:p^k-1
        rhs = Rn(a*(trace-a) - det)
        for b in 1:(p^k - 1)
            if b % p != 0
                c = Rk(rhs / b)
                matrices_dict[(Rk(a), Rk(b), c)] = get(matrices_dict, (Rk(a), Rk(b), c), 0) + p^(2*(n-k))
            end
        end
    end

    for a in Rn
        rhs = a*(trace-a) - det
        if rhs != 0
            pval = valuation(lift(rhs), p)
            for i in 1:pval
                if n-i >= k
                    add_to_dict1(p, n, k, i, rhs / p^i, matrices_dict, a, k, Rk)
                else
                    add_to_dict1(p, n, k, i, rhs / p^i, matrices_dict, a, n-i, Rk)
                end
            end
        else 
            for c in 0:(p^k - 1)
                matrices_dict[(Rk(a), zero(Rk), Rk(c))] = get(matrices_dict, (Rk(a), zero(Rk), Rk(c)), 0) + p^(n-k)
            end
            for b in p:p:(p^k - 1)
                pval = valuation(b, p)
                add_to_dict2(p, n, k, pval, matrices_dict, a, b, p^(pval + n - k), p^(2*(n-k)), Rk)
            end
            for pval in k:(n - 1)
                add_to_dict2(p, n, k, pval, matrices_dict, a, 0, p^pval * (p^(n-pval) - p^(n-pval-1)), p^(n-k) * (p^(n-pval) - p^(n-pval-1)), Rk)
            end
        end
    end
    return matrices_dict
end

build_matrices_ratio3 (generic function with 1 method)

In [87]:
# compute ratio3 for specific k and n
function compute_ratio3_singleK(p, n, k, baseM, max_e)
    trace, det = baseM[1]+baseM[4], baseM[1]*baseM[4] - baseM[2]*baseM[3]
    
    equations_template = Vector{MatElem}()
    Rk, = residue_ring(ZZ, p^k)
    for a in 0:(p^k-1)
        d = baseM[1]+baseM[4] - a
        equations = matrix(Rk, [
            [a - baseM[1], -baseM[3], 0, 0],
            [0, 0, d - baseM[1], -baseM[3]],
            [-baseM[2], a - baseM[4], 0, 0],
            [0, 0, -baseM[2], d - baseM[4]]
        ])
        push!(equations_template, equations)
    end

    matrices = build_matrices_ratio3(p, n, trace, det, k)
    res = ratio3(p, n, k, equations_template, matrices)
    ratio3_val = Dict{Int, Int}()
    for e in 0:max_e
        ratio3_val[e] = 0
    end
    for (M, minVal) in res
        for e in max_e:-1:minVal
            ratio3_val[e] += matrices[M]
        end
    end
        
    ratio3_res = [rationalize(ratio3_val[e] / sl2size(p, n)) for e in 0:max_e]
    println("ratio 3:")
    for (e, row) in enumerate(ratio3_res)
        println("e = $(e - 1) → ", row)
    end
end

@time compute_ratio3_singleK(3, 5, 5, (1, 0, 0, 4), 6)

ratio 3:
e = 0 → 1//6
e = 1 → 7//6
e = 2 → 3//2
e = 3 → 3//2
e = 4 → 3//2
e = 5 → 3//2
e = 6 → 3//2
  0.845925 seconds (4.32 M allocations: 352.439 MiB, 33.36% gc time)
